# ASHES TMPSF Timeseries Exploration

**Spec**: `specs/001-ashes-tmpsf-timeseries/spec.md`

**Research Question**: How does diffuse hydrothermal vent temperature vary as a function of time for the year 2018 at the ASHES vent field?

**Data**: OOI TMPSF sensor (RS03ASHS-MJ03B-07-TMPSFA301) - NetCDF files, 24 thermistor channels

**Output**: Daily average temperature timeseries for 2018

In [ ]:
# Setup: Import packages and define paths
import xarray as xr
import pandas as pd
import numpy as np
import hvplot.pandas
import hvplot.xarray
from pathlib import Path

# Data paths
DATA_DIR = Path('/home/jovyan/ooi/kdata/RS03ASHS-MJ03B-07-TMPSFA301-streamed-tmpsf_sample')
OUTPUT_DIR = Path('/home/jovyan/my_data/axial/axial_tmpsf')
FIGURE_DIR = Path('../outputs/figures')

# Analysis parameters - 2018 only
START_DATE = '2018-01-01'
END_DATE = '2018-12-31'

print(f'Data directory: {DATA_DIR}')
print(f'Number of NetCDF files: {len(list(DATA_DIR.glob("*.nc")))}')
print(f'Analysis period: {START_DATE} to {END_DATE}')

In [ ]:
# Load Data: Open all NetCDF files with xarray
# Using open_mfdataset for multi-file loading with lazy evaluation

# Get list of NetCDF files
nc_files = sorted(DATA_DIR.glob('*.nc'))
print(f'Found {len(nc_files)} NetCDF files')

# Load all files - this uses lazy loading (dask) by default
# combine='nested' and concat_dim='obs' to handle the OOI data structure
print('Loading dataset (lazy)...')
ds = xr.open_mfdataset(
    nc_files,
    combine='nested',
    concat_dim='obs',
    parallel=True
)

# Swap dims from 'obs' to 'time' for easier time-based operations
ds = ds.swap_dims({'obs': 'time'})
print(f'Dataset loaded: {ds.dims}')

In [ ]:
# Select temperature variables and QC flags
# Temperature channels: temperature01 through temperature24
# QC flags: temperatureNN_qartod_results

temp_vars = [f'temperature{i:02d}' for i in range(1, 25)]
qc_vars = [f'temperature{i:02d}_qartod_results' for i in range(1, 25)]

# Check which variables exist in the dataset
available_temp = [v for v in temp_vars if v in ds.data_vars]
available_qc = [v for v in qc_vars if v in ds.data_vars]

print(f'Temperature variables found: {len(available_temp)}/24')
print(f'QC flag variables found: {len(available_qc)}/24')

# Select only the variables we need (temperature + time + QC flags)
vars_to_keep = ['time'] + available_temp + available_qc
ds_selected = ds[available_temp + available_qc]
print(f'Selected dataset variables: {len(ds_selected.data_vars)}')

In [ ]:
# Filter to 2018
print(f'Original time range: {ds_selected.time.min().values} to {ds_selected.time.max().values}')

ds_filtered = ds_selected.sel(time=slice(START_DATE, END_DATE))

print(f'Filtered time range: {ds_filtered.time.min().values} to {ds_filtered.time.max().values}')
print(f'Number of observations: {ds_filtered.dims["time"]}')

## QC: Data Loading Verification

Verify that:
1. Time range covers 2018
2. All 24 temperature channels are present

In [ ]:
# QC: Verify time range and temperature channels

# T007: Check time range is 2018
time_min = pd.Timestamp(ds_filtered.time.min().values)
time_max = pd.Timestamp(ds_filtered.time.max().values)
expected_start = pd.Timestamp('2018-01-01')
expected_end = pd.Timestamp('2018-12-31')

assert time_min.year == 2018, f'Time range does not start in 2018: {time_min}'
assert time_max.year == 2018, f'Time range does not end in 2018: {time_max}'
print(f'✓ T007: Time range verified: {time_min.date()} to {time_max.date()}')

# T008: Check all 24 temperature channels
assert len(available_temp) == 24, f'Missing temperature channels: found {len(available_temp)}/24'
print(f'✓ T008: All 24 temperature channels present')

print('\nPhase 2 checkpoint passed: Data loaded and filtered to 2018')

## Phase 3: QC Assessment

Examine QARTOD flags and filter suspect data.

In [ ]:
# Examine QARTOD flag distribution
# QARTOD flags: 1=pass, 2=not evaluated, 3=suspect, 4=fail, 9=missing

if available_qc:
    # Check flag distribution for first temperature channel
    qc_var = available_qc[0]
    print(f'Examining QC flags for {qc_var}:')
    
    # Load a sample of QC flags
    qc_sample = ds_filtered[qc_var].values
    unique, counts = np.unique(qc_sample[~np.isnan(qc_sample)], return_counts=True)
    
    flag_meanings = {1: 'pass', 2: 'not evaluated', 3: 'suspect', 4: 'fail', 9: 'missing'}
    for flag, count in zip(unique, counts):
        pct = 100 * count / len(qc_sample)
        meaning = flag_meanings.get(int(flag), 'unknown')
        print(f'  Flag {int(flag)} ({meaning}): {count:,} ({pct:.1f}%)')
else:
    print('No QARTOD QC flags found in dataset')

## Phase 4: DataFrame Creation & Daily Averaging

Convert to pandas DataFrame and compute daily averages.

In [ ]:
# Convert to pandas DataFrame
# Select only temperature variables (exclude QC flags for now)
print('Converting to DataFrame...')
df = ds_filtered[available_temp].to_dataframe()
print(f'DataFrame shape: {df.shape}')
print(f'Columns: {list(df.columns)}')

In [ ]:
# Compute daily average temperature
# Average across all 24 thermistor channels, then resample to daily

# First compute mean across all temperature channels for each timestamp
df['temp_mean'] = df[available_temp].mean(axis=1)

# Resample to daily averages
df_daily = df['temp_mean'].resample('D').mean().to_frame()
df_daily.columns = ['daily_avg_temp']

print(f'Daily averages shape: {df_daily.shape}')
print(f'Date range: {df_daily.index.min().date()} to {df_daily.index.max().date()}')
print(f'Number of days: {len(df_daily)}')
df_daily.head()

In [ ]:
# T017 QC: Verify daily averages DataFrame
# 2018 has 365 days

expected_days = 365
actual_days = len(df_daily)

print(f'Expected days: {expected_days}')
print(f'Actual days: {actual_days}')

if actual_days == expected_days:
    print(f'✓ T017: Daily averages has {actual_days} rows (full year)')
else:
    missing = expected_days - actual_days
    print(f'⚠ T017: Missing {missing} days ({actual_days}/{expected_days})')
    
# Check for NaN values
nan_count = df_daily['daily_avg_temp'].isna().sum()
print(f'Days with NaN: {nan_count}')

## Phase 5: Visualization

Interactive timeseries plot of daily average temperature for 2018.

In [ ]:
# Create interactive hvplot timeseries
plot = df_daily.hvplot.line(
    y='daily_avg_temp',
    title='ASHES Vent Field - Daily Average Temperature (2018)',
    xlabel='Date',
    ylabel='Temperature (°C)',
    width=900,
    height=400,
    grid=True
)
plot

In [ ]:
# T022 QC: Verify plot displays 2018 time range
print('Plot verification:')
print(f'  Start date: {df_daily.index.min().date()}')
print(f'  End date: {df_daily.index.max().date()}')
print(f'  Temperature range: {df_daily["daily_avg_temp"].min():.2f}°C to {df_daily["daily_avg_temp"].max():.2f}°C')
print(f'  Mean temperature: {df_daily["daily_avg_temp"].mean():.2f}°C')

if df_daily.index.min().year == 2018 and df_daily.index.max().year == 2018:
    print('✓ T022: Plot displays 2018 time range')

In [ ]:
# Optional: Save outputs
# Uncomment to save DataFrame and plot

# Save daily averages to parquet
# OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
# df_daily.to_parquet(OUTPUT_DIR / 'tmpsf_2018_daily.parquet')
# print(f'Saved DataFrame to {OUTPUT_DIR / "tmpsf_2018_daily.parquet"}')

# Save plot to HTML
# FIGURE_DIR.mkdir(parents=True, exist_ok=True)
# hvplot.save(plot, FIGURE_DIR / 'tmpsf_2018_daily.html')
# print(f'Saved plot to {FIGURE_DIR / "tmpsf_2018_daily.html"}')